#### ParameterGrid 기초

In [2]:
from sklearn.model_selection import ParameterGrid

grid = {"n_neighbors": [3, 5, 7],
        "metric": ["Manhattan", "Euclidean"]}

#ParameterGrid(grid)
list(ParameterGrid(grid))

[{'metric': 'Manhattan', 'n_neighbors': 3},
 {'metric': 'Manhattan', 'n_neighbors': 5},
 {'metric': 'Manhattan', 'n_neighbors': 7},
 {'metric': 'Euclidean', 'n_neighbors': 3},
 {'metric': 'Euclidean', 'n_neighbors': 5},
 {'metric': 'Euclidean', 'n_neighbors': 7}]

#### 함수의 입력으로 사전을 입력받기: **의 사용

In [3]:
def f(a, b):
    return a + b

input_f = {"a": 1, "b": 2}
#input_f = {"b": 2, "a": 1}
f(**input_f)

3

#### 최대값 찾기 알고리즘 예시

In [7]:
L = [10, 20, 30, 10, 20]
max_value = -999999  # 매우 작은 값으로 설정
for value in L:    # Value_list를 순서대로 탐색
    print(value, max_value)
    if max_value < value:
        max_value = value # 탐색한 값이 현재 최대 값보다 크면 업데이트

print(max_value)

10 -999999
20 10
30 20
10 30
20 30
30


#### 최소값 찾기 알고리즘 예시

In [8]:
L = [10, 20, 30, 10, 20]
min_value = 999999  # 매우 큰 값으로 설정
for value in L:    # Value_list를 순서대로 탐색
	if min_value > value:
		min_value = value # 탐색한 값이 현재 최소 값보다 작으면 업데이트

print(min_value)

10


#### 그리드 서치 실습 예제
- 사용 데이터: iris dataset (sklearn 제공 데이터)<br><br>
- 사용 모델: (1) k-최근접 이웃
    - n_neighbors (3, 5, 7)
    - metric (euclidean, manhattan)<br><br>
- 사용 모델: (2) 서포트 벡터 머신
    - kernel: rbf, linear
    - C: 0.1, 1, 10
<br><br>
- 평가 척도: F1 score

In [9]:
# 예제 데이터 불러오기
from sklearn.datasets import load_iris
X = load_iris()['data'] # feature
Y = load_iris()['target'] # label

In [10]:
# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [11]:
Train_X.shape

(112, 4)

In [12]:
Test_X.shape

(38, 4)

In [7]:
# 모델 불러오기
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC

In [8]:
# 파라미터 그리드 생성
param_grid = dict() 
# 입력: 모델 함수, 출력: 모델의 하이퍼 파라미터 그리드

# 모델별 파라미터 그리드 생성
param_grid_for_knn = ParameterGrid({"n_neighbors": [3, 5, 7],
                           "metric":['euclidean', 'manhattan']})

param_grid_for_svm = ParameterGrid({"C": [0.1, 1, 10],
                           "kernel":['rbf', 'linear']})

# 모델 - 하이퍼 파라미터 그리드를 param_grid에 추가
param_grid[KNN] = param_grid_for_knn
param_grid[SVC] = param_grid_for_svm

In [9]:
# 하이퍼 파라미터 튜닝 
best_score = -1 # 현재까지 찾은 가장 높은 f1_score (f1 score는 절대 0보다 작을수 없기에, -1로 설정해도 무방)

from sklearn.metrics import f1_score

for model_func in [KNN, SVC]:
    for param in param_grid[model_func]:
        model = model_func(**param).fit(Train_X, Train_Y)
        pred_Y = model.predict(Test_X)        
        score = f1_score(Test_Y, pred_Y, average = 'micro')
        
        if score > best_score: 
            # 현재 점수가 지금까지 찾은 최고 점수보다 좋으면, 최고 모델, 파라미터, 점수 업데이트
            best_model_func = model_func
            best_score = score
            best_param = param
            
            # best_model = model

In [10]:
print("모델:", best_model_func)
print("점수:", best_score)
print("파라미터:", best_param)

모델: <class 'sklearn.svm._classes.SVC'>
점수: 0.9736842105263158
파라미터: {'C': 1, 'kernel': 'rbf'}


In [11]:
# 최종 모델 학습: 전체 X와 전체 Y에 대해.
final_model = best_model_func(**best_param).fit(X, Y)